In [32]:
import requests
import json
import pandas as pd
data = requests.get("https://data.ffvl.fr/api/?base=terrains&mode=json&key=00000000000000000000000000000000").text
weather_data = json.loads(data)
global villedata
geo_data = pd.read_csv("data/cities.csv")
geo_data.head()




,insee_code,city_code,zip_code,label,latitude,longitude,department_name,department_number,region_name,region_geojson_name
0,25620,ville du pont,25650,ville du pont,46.999873,6.498147,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
1,25624,villers grelot,25640,villers grelot,47.361512,6.235167,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
2,25615,villars les blamont,25310,villars les blamont,47.368384,6.871415,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
3,25619,les villedieu,25240,les villedieu,46.713906,6.265831,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
4,25622,villers buzon,25170,villers buzon,47.228558,5.852187,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté


In [33]:
def get_department_region(city_name, geo_data):
    # Normalize the city names to lowercase for matching
    geo_data['label'] = geo_data['label'].str.lower()
    city_name = city_name.lower()
    
    # Find the row where the city name matches
    matching_row = geo_data[geo_data['label'] == city_name]
    
    # If there's a match, return the department and region
    if not matching_row.empty:
        return {
            'department_name': matching_row.iloc[0]['department_name'],
            'department_number': matching_row.iloc[0]['department_number'],
            'region_name': matching_row.iloc[0]['region_name']
        }
    else:
        # If there's no match, return None or an empty dictionary
        return None
SOLAR_LATITUDE_THRESHOLD = 45.0  # Arbitrary value for illustration
WIND_SPEED_THRESHOLD = 5  # Assuming wind orientations contain a numeric speed indicator, this is arbitrary

def score_for_wind_energy(weather):
    # Assume that more wind orientations and higher altitude are better for wind energy
    wind_score = 0
    wind_ok = weather.get('wind_orientations_ok')
    if wind_ok:
        wind_conditions = wind_ok.split(';')[:-1]  # Remove the empty string due to trailing ;
        wind_score = len(wind_conditions)  # More orientations mean better wind potential
    altitude = int(weather.get('altitude', 0))
    wind_score += altitude / 100  # Higher altitude might be better for wind energy
    return wind_score

def score_for_solar_energy(weather):
    # Assume regions with lower latitudes have better solar potential due to higher solar irradiance.
    latitude = float(weather.get('latitude', 0))
    
    # A simple linear scoring based on latitude, where the score decreases as latitude increases.
    # For example, we can assume the maximum score for latitude 0 and decrease linearly until the SOLAR_LATITUDE_THRESHOLD.
    max_solar_score = 10  # Maximum score at the equator (latitude 0)
    score_per_degree = max_solar_score / SOLAR_LATITUDE_THRESHOLD  # Score decrease per degree of latitude
    
    # Calculate the solar score based on latitude, with higher latitudes receiving lower scores.
    solar_score = max(0, max_solar_score - (latitude * score_per_degree))
    
    return solar_score


# Apply the scoring functions to each region and create a list of dictionaries for DataFrame
energy_data_list = []
for region in weather_data:
    # Get department and region info
    dept_region_info = get_department_region(region['city'], geo_data)
    
    # Update region with department and region info
    region.update(dept_region_info if dept_region_info else {'department_name': "", 'department_number': "", 'region_name': ""})
    
    # Calculate energy scores
    region['wind_energy_score'] = score_for_wind_energy(region)
    region['solar_energy_score'] = score_for_solar_energy(region)
    
    # Determine best green energy
    region['best_green_energy'] = 'wind' if region['wind_energy_score'] > (region['solar_energy_score'] * 100) else 'solar'
    
    # Append the data to the list
    energy_data_list.append(region)

# Create a DataFrame and remove duplicates
energy_scores_df = pd.DataFrame(energy_data_list).drop_duplicates(subset=['city'])

# Display the DataFrame (optional)
energy_scores_df.head()
#energy_scores_df.to_excel("Test.xlsx")


,suid,toponym,zip,city,possible_usages,possible_usages_text,flying_functions,flying_functions_text,emergency_contacts,emergency_services_text,...,handi_access_possible,terrain_definition_polygon,stru_cp,stru_ligue,department_name,department_number,region_name,wind_energy_score,solar_energy_score,best_green_energy
0,8,ECHENOZ-LA-MELINE - BOIS DE BREULEUX,'70000,ECHENOZ LA MELINE,3,parapente;delta;,1,décollage;,1,mobile;vhf;,...,0,None,'70000,06 - LIGUE DE VOL LIBRE BOURGOGNE FRANCHE COMTE,haute-saône,70,bourgogne-franche-comté,3.63,0.000000,wind
2,10,CORDES SAINT MARCEL CAMPES TEST,'81170,ST MARCEL DE CAMPES,3,parapente;delta;,1,décollage;,1,mobile;vhf;,...,0,None,'81400,28 - LIGUE DE VOL LIBRE OCCITANIE,,,,4.91,0.208044,solar
4,12,LA-BELLE-IDEE,'77750,BASSEVELLE,3,parapente;delta;,8,treuil;,0,None,...,0,None,'78170,05 - LIGUE DE VOL LIBRE DE PARIS ILE DE FRANCE,seine-et-marne,77,île-de-france,8.95,0.000000,wind
5,15,LEZAT - BELVEDERE,'39400,LEZAT,3,parapente;delta;,1,décollage;,1,mobile;vhf;,...,1,None,'39400,06 - LIGUE DE VOL LIBRE BOURGOGNE FRANCHE COMTE,,,,9.99,0.000000,wind
7,17,NOIRMONT - PISTE NOIRE N°1,'39220,LES ROUSSES,7,parapente;delta;speed-riding;,1,décollage;,1,mobile;vhf;,...,0,None,'39400,06 - LIGUE DE VOL LIBRE BOURGOGNE FRANCHE COMTE,jura,39,bourgogne-franche-comté,14.95,0.000000,wind


In [34]:
import folium
from folium.plugins import MarkerCluster
from branca.colormap import linear

m = folium.Map(location=[46.2276, 2.2137], zoom_start=6)

# Define a marker cluster
marker_cluster = MarkerCluster().add_to(m)

# Define color maps
wind_colormap = linear.YlGnBu_09.scale(min(energy_scores_df['wind_energy_score']), max(energy_scores_df['wind_energy_score']))
solar_colormap = linear.OrRd_09.scale(min(energy_scores_df['solar_energy_score']), max(energy_scores_df['solar_energy_score']))

# Add points to the map
for idx, row in energy_scores_df.iterrows():
    if row['best_green_energy'] == 'wind':
        color = wind_colormap(row['wind_energy_score'])
    else:
        color = solar_colormap(row['solar_energy_score'])
    
    folium.CircleMarker(
        location=(row['latitude'], row['longitude']),
        radius=10,
        popup=f"{row['city']}: {row['best_green_energy']}",
        color=color,
        fill=True,
        fill_color=color
    ).add_to(marker_cluster)

# Add color map legend
wind_colormap.caption = 'Wind Energy Score'
wind_colormap.add_to(m)

solar_colormap.caption = 'Solar Energy Score'
solar_colormap.add_to(m)

# Save to HTML
map_file_path = 'results/energy_map.html'
m.save(map_file_path)

map_file_path

'energy_map.html'

In [35]:
# geojson_url = "https://france-geojson.gregoiredavid.fr/repo/regions.geojson"
# geojson_regions = json.loads(requests.get(geojson_url).text)


# # Calculate average scores by region
# average_scores = energy_scores_df.groupby('region_name').mean().reset_index()

# # Function to determine the color based on the dominant energy type
# def get_color(feature):
#     region_name = feature['properties']['nom']
    
#     # Ensure we compare the lowercase versions of the region names
#     matching_row = average_scores[average_scores['region_name'].str.lower() == region_name.lower()]
    
#     if not matching_row.empty:
#         # Calculate the percentage of wind energy score
#         total_score = matching_row.iloc[0]['wind_energy_score'] + matching_row.iloc[0]['solar_energy_score']
#         wind_percentage = (matching_row.iloc[0]['wind_energy_score'] / total_score) * 100 if total_score > 0 else 0
        
#         # Color the region blue if wind energy score is more than 70% of the total, otherwise yellow
#         if wind_percentage > 98:
#             return 'blue'
#         else:
#             return 'yellow'
#     else:
#         return 'grey'  # Default color if no match is found


# # Create a base map
# m = folium.Map(location=[46.2276, 2.2137], zoom_start=6)

# # Add the GeoJSON overlay
# folium.GeoJson(
#     geojson_regions,
#     style_function=lambda feature: {
#         'fillColor': get_color(feature),
#         'color': 'black',
#         'weight': 1,
#         'fillOpacity': 0.7,
#     }
# ).add_to(m)

# # Save to HTML
# map_file_path = 'energy_map_regions.html'
# m.save(map_file_path)

# print("Map saved to:", map_file_path)

In [36]:
# # Load the GeoJSON for French departments
# geojson_url = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
# geojson_departments = json.loads(requests.get(geojson_url).text)



# # Calculate average scores by department
# average_scores = energy_scores_df.groupby('department_number').mean().reset_index()

# # Function to determine the color based on the dominant energy type
# def get_color(feature):
#     department_number = feature['properties']['code']
    
#     # Ensure we compare the department numbers as the same type (both as strings or both as integers)
#     matching_row = average_scores[average_scores['department_number'].astype(str) == str(department_number)]
    
#     if not matching_row.empty:
#         # Calculate the percentage of wind energy score
#         total_score = matching_row.iloc[0]['wind_energy_score'] + matching_row.iloc[0]['solar_energy_score']
#         wind_percentage = (matching_row.iloc[0]['wind_energy_score'] / total_score) * 100 if total_score > 0 else 0
        
#         # Color the department blue if wind energy score is more than 70% of the total, otherwise yellow
#         if wind_percentage > 98:
#             return 'blue'
#         else:
#             return 'yellow'
#     else:
#         return 'grey'  # Default color if no match is found

# # Create a base map
# m = folium.Map(location=[46.2276, 2.2137], zoom_start=6)

# # Add the GeoJSON overlay
# folium.GeoJson(
#     geojson_departments,
#     style_function=lambda feature: {
#         'fillColor': get_color(feature),
#         'color': 'black',
#         'weight': 1,
#         'fillOpacity': 0.7,
#     }
# ).add_to(m)

# # Save to HTML
# map_file_path = 'energy_map_departments.html'
# m.save(map_file_path)

# print("Map saved to:", map_file_path)
